# Excercise 1

Implement integration over a rectangular domain with a double Legendre quadrature.

In [44]:
import math
from scipy.special import roots_legendre

def legendre(ax, bx, ay, by, m, n, f):
    #xx = [  for k in range(2, m)] #approximately
    #yy = [  for k in range(2, n)]
    #wx = [  for k in range(2, m)]
    #wy = [  for k in range(2, n)]
    xx, wx = roots_legendre(m)
    yy, wy = roots_legendre(n)
    
    v = 0
    
    for i in range(m):
        for j in range(n):
            v += f((bx - ax) / 2 * xx[i] + (ax + bx) / 2, (by - ay) / 2 * yy[i] + (ay + by) / 2) * wx[i] * wy[j]
    
    return v

def f(x, y):
    return x + y

legendre(0, 2, 0, 2, 2, 2, f)

8.0

# Excercise 2

Implement integration over a triangular domain with a mixed Legendre/Jacobi quadrature. 

In [61]:
import math
from scipy.special import roots_sh_legendre, roots_jacobi
from numpy import allclose

def jacobi_legendre(ax, bx, ay, m, n, f):
    tt, wt = roots_sh_legendre(m)
    xx, wx = roots_jacobi(n, 0, 1)
    
    v = 0
    
    for i in range(m):
        for j in range(n):
            v += f(0.5 + 0.5 * xx[j], tt[i] * (0.5 + 0.5 * xx[j])) * wt[i] * wx[j]
    
    return v / 4

def f0(x, y):
    return 1

def f1(x, y):
    return x + 2*y

def f2(x, y):
    return x**2 + y**2 + 1

answer = {f0: 0.5, f1: 2/3, f2: 5/6}
for f in [f0, f1, f2]:
    print(abs(jacobi_legendre(0, 1, 0, 10, 10, f) - answer[f]))

2.220446049250313e-16
1.1102230246251565e-16
4.440892098500626e-16


# Excercise 5

Use the Sobol sequence to calculate the value of
$$
\iint\limits_D e^{-(x^2 + y^2)} \, dx\, dy  \;,
$$
where $D$ is the unit square. Compare to the exact answer (which can be expressed in terms of the error function).

In [30]:
import numpy as np
import sobol_seq
from scipy.special import erf

rndm = np.random.RandomState(12345)

def func(x, y):
    return np.exp(- x**2 - y**2)

def inside(x, y):
    return (x <= 1)*(y <= 1)

def sample(N):
    N = int(N)
    x = sobol_seq.i4_sobol_generate(2, N)
    accepted = inside(x[:, 0], x[:, 1])
    denom = np.count_nonzero(accepted)
    return np.sum(func(x[accepted, 0], x[accepted, 1]) / N), denom

answer = np.pi / 4 * (erf(1))**2

for n in [10, 100, 1000, 10000, 1e5, 1e6]:
    res = sample(n)
    err = res[0] - answer
    print("%8d  %.7g  %.7g %g" % (n, res[0], answer, err))

      10  0.5591819  0.5577463 0.00143559
     100  0.5599091  0.5577463 0.00216285
    1000  0.5579124  0.5577463 0.000166136
   10000  0.5578148  0.5577463 6.85321e-05
 1000000  0.5577471  0.5577463 7.80122e-07
10000000  0.5577464  0.5577463 9.73227e-08
